In [1]:
from text2brick.models import GraphLegoWorldData
import numpy as np
from text2brick.dataset.dataset import Dataset

In [2]:
table = np.array([[1, 1, 1, 1],
                  [1, 1, 1, 1],
                  [1, 1, 0, 0],
                  [1, 1, 1, 1]])

In [3]:
dataset = Dataset()
array, _, _, _ = dataset.sample(sample_index=1)

In [4]:
graph = GraphLegoWorldData(img=table)
graph.print_graph()

Node 0: {'x': 0, 'y': 0, 'validity': True}
Node 1: {'x': 2, 'y': 0, 'validity': True}
Node 2: {'x': 0, 'y': 1, 'validity': True}
Node 3: {'x': 0, 'y': 2, 'validity': True}
Node 5: {'x': 0, 'y': 3, 'validity': True}
Edge (0, 2): {}
Edge (2, 3): {}
Edge (3, 5): {}


In [5]:
vars(graph.graph)

{'graph': {},
 '_node': {0: {'x': 0, 'y': 0, 'validity': True},
  1: {'x': 2, 'y': 0, 'validity': True},
  2: {'x': 0, 'y': 1, 'validity': True},
  3: {'x': 0, 'y': 2, 'validity': True},
  5: {'x': 0, 'y': 3, 'validity': True}},
 '_adj': {0: {2: {}}, 1: {}, 2: {0: {}, 3: {}}, 3: {2: {}, 5: {}}, 5: {3: {}}},
 '__networkx_cache__': {},
 'nodes': NodeView((0, 1, 2, 3, 5)),
 'edges': EdgeView([(0, 2), (2, 3), (3, 5)])}